In [ ]:
import torch
import numpy as np
import random, os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
def seed_it(seed):
    random.seed(seed)
    os.environ["PYTHONSEED"] = str(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    torch.manual_seed(seed)
seed_it(42)

In [ ]:
from datasets import load_dataset

datasets = ["dureader"]

for dataset in datasets:
    dureader_data = load_dataset('THUDM/LongBench', dataset, split='test')

In [ ]:
dataset_seed = 42
seed_it(dataset_seed)
all_index = list(range(len(dureader_data)))
train_index = np.sort(random.sample(all_index, int(len(all_index)*0.8)))
test_index = np.sort(list(set(all_index).difference(set(train_index))))
print(f'prepare dataset, train size: {len(train_index)}, test size: {len(test_index)}')

In [ ]:
import re

def extract_passages(text):
    pattern = r'文章\d+\n(.*?)(?=\文章\d+|\Z)'
    passages = re.findall(pattern, text, flags=re.DOTALL)
    passages_data = []
    for passage in passages:
        p = passage.strip().split('\n')
        if len(p) != 2:
            print(p)
            raise ValueError()
        passages_data.append({'title': p[0].replace('标题：', '').strip(), 'text': p[1].strip()})
    return passages_data

text = dureader_data[0]['context']
passages = extract_passages(text)

len(passages)

In [ ]:
dureader_data[0]

{'input': '热诚传说结局',
 'context': '文章1\n标题：对结局好失望_热诚传说x吧_百度贴吧\n我想看的不是百合搞基番!为什么看公主和男主在一起,等了这么久的大结局,然后……你给我看这个?感觉前面都白看了,早知道不知结局了,前面25集都挺好的!看到大结局好失落,女主们最终还是没有第到使雷回来,新的道师也出现了,而男主醒来应该是几百年后了(城市都现代化了)!公主早就死了几百年了吧 不楼主请仔细看 新的导师就是史雷 史雷醒来应该也是几年后的事情 然而她们都长大了 下载贴吧APP看高清直播、视频! 贴吧页面意见反馈 违规贴吧举报反馈通道 贴吧违规信息处理公示 内容:\n\n文章2\n标题：《热诚传说》动画的结局是百合??? - 動漫區 - 百合会 - Powered by \n您需要 登录 才可以下载或查看,没有帐号?成为会员 四月百合淡季什么的,不存在的,顺便快去看re:creator最新一集 真的吗?真的吗!快来人打醒我 补特定几集就行了,整部作品来说没有明显的感情线,大家之间的距离都保持的很好,最后倒是真官方盖章百合,推得丧心病狂 这结局真喷了。。。但是为了百合补这番还是算了吧,不值得 ……好像可以借着这个结局坑别人玩一波游戏原作了。 参与人数 1积分+1 收起 理由 坟头纳彩 + 1 稳!“公主呢?” 查看全部评分 求补番指南,打算挑着看一看 您需要登录后才可以回帖 登录 | 成为会员\n\n文章3\n标题：【剧透】通关剧情分析吐槽 - 热情传说 - A9VG电玩部落论坛\n精华 0 帖子 1006 威望 0点 积分 1089 点 注册时间 2005-6-12 最后登录 2017-6-9 精华 0 帖子 8 威望 0点 积分 8点 注册时间 2014-9-7 最后登录 2017-3-16 表示无比赞同!玩了后说不出的感觉,就是感觉稀里糊涂+平淡无奇,说差也不差,但也说不上好。本来传说系列的一大特色就是充满矛盾、激情和冲突的剧情,到后期或结局往往令人心潮澎湃或者感动不已,远的如永恒传说,宿命传说2,近的如TOX。蛮失望的,不过会继续追传说系列! 您需要登录后才可以回帖 登录 | 注册\n\n文章4\n标题：热诚传说X 剧情介绍(全集大结局)_穿帮网\nchecking... 概述 简介 海报 我来发布 2016年放送開始!制作はufotab

In [ ]:
from copy import deepcopy
from tqdm import tqdm
dureader_dataset = []
for data in tqdm(dureader_data):
    data = deepcopy(data)
    passages = extract_passages(data['context'])
    data['passages'] = passages
    dureader_dataset.append(data)

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:00<00:00, 1427.27it/s]


In [ ]:
dureader_dataset[12]

{'input': '遮光垫有用吗',
 'context': '文章1\n标题：大家觉得中控台遮光垫有用吗;【瑞纳吧】_百度贴吧\n我感觉没大用,只是个装饰呗 感觉按上了以后,显得仪表台有些乱。有一个实验说,先往一个瓶子里装满小石块,然后还能装下细沙,最后看似满了,却还能装下半瓶水。这个实验告诉我们,吃自助餐一定不能先喝饮料 我也弄了一个。感觉就是夏天能不让太阳把工作台晒的很热。其他没用了 噢,我打算买个来。看样不用买了 问的都是废话!当然有用了! 给你讲点易懂得的 1 是黑色吧 最吸热的颜色 2 大家都知道热量传递 是高的传向低的 那么问题来了 避光垫所吸收的热量 是怎么散发的? 很显然 到一定温度 回传向工作台 那么就等于 没有必要避光了工作台本身就是深色 反光小 所以没必要 买这个 你觉得有用么 毫无用处 用了前档玻璃不反光看的清楚点!其他也没什么! 防反光的,有用, 来了来了来了了来了来了啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦 我买来铺了,最大的用出就是能接尘土,还能防反光,隔热绝对无用,本身垫子是黑色就吸热,再盖上这垫子根本不散热,晒完摸仪表台烫死人,如果只用玻璃的遮阳帘就可以了 多少有点用,但是太难看了,最好不要 绝对有用,前玻璃完全不反光,眼睛很舒服,前台也不烫手,虽然垫子下面温度一样,但垫子上根本不烫,有小孩的话很必要 我没装,并没有什么卵用,还不好看 22楼正解。。仪表台热。但是垫子并不热。。反光什么的。没用过的就别说没用。反正我用了,很好。拿掉的话很不习惯。说乱的,并没有。款式的问题 个人感觉,有用!车里味道清了。\n\n文章2\n标题：【图】遮光垫是保护中控还是毁坏中控??大家引以为戒吧!!!_马自达CX-\n现在商家已经推请家告知该如何维权当然如果知道怎么能处理哥们也请赶紧赐教 [ 本帖最后由 火星大虾 于 2014-10-28 12:28:22 编辑 ] 沙发不留了 闹心~~~ 也许暴晒几天自己就好了。 不过这个遮光垫放车上真是感觉土到想死啊。不知道什么样的人想出来的东东。 bigflfl 2014/10/27 22:11:57 发表在 板凳 也许暴晒几天自己就好了。 揭开遮光垫一段时间了 没变化啊\n\n文章3\n标题：买遮光垫有必要吗?_马自达CX-5论坛_太平洋汽车论坛 \n您好,精华帖至少要有15张图片,文字不少200个字!并且是原

In [ ]:
passages[0]

{'title': '标题：对结局好失望_热诚传说x吧_百度贴吧',
 'text': '我想看的不是百合搞基番!为什么看公主和男主在一起,等了这么久的大结局,然后……你给我看这个?感觉前面都白看了,早知道不知结局了,前面25集都挺好的!看到大结局好失落,女主们最终还是没有第到使雷回来,新的道师也出现了,而男主醒来应该是几百年后了(城市都现代化了)!公主早就死了几百年了吧 不楼主请仔细看 新的导师就是史雷 史雷醒来应该也是几年后的事情 然而她们都长大了 下载贴吧APP看高清直播、视频! 贴吧页面意见反馈 违规贴吧举报反馈通道 贴吧违规信息处理公示 内容:'}

In [ ]:
raw_data_path = r'dureader_raw\preprocessed\devset\search.dev.json'  # load raw dataset
import json
dureader_raw_data = []
with open(raw_data_path, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        data = json.loads(line)
        dureader_raw_data.append(data)
    f.close()

In [ ]:
len(dureader_raw_data), dureader_raw_data[0].keys(), dureader_raw_data[0]['documents'][0].keys()

(5000,
 dict_keys(['documents', 'answer_spans', 'answer_docs', 'fake_answers', 'question', 'segmented_answers', 'answers', 'entity_answers', 'segmented_question', 'question_type', 'match_scores', 'fact_or_opinion', 'question_id']),
 dict_keys(['is_selected', 'title', 'most_related_para', 'segmented_title', 'segmented_paragraphs', 'paragraphs']))

In [ ]:
def search_raw_data(question):
    for raw_data in dureader_raw_data:
        raw_question = raw_data['question'].strip()
        if question == raw_question:
            return raw_data
    raise ValueError()

In [ ]:
dureader_dataset_2 = []
count = []
for data in tqdm(dureader_dataset[:]):
    data = deepcopy(data)
    question = data['input']
    # search raw data
    raw_data = search_raw_data(question)
    gt_documents = []
    for doc in raw_data['documents']:
        if doc['is_selected']:
            gt_documents.append(doc)
    gt_titles = [doc['title'].strip() for doc in gt_documents]
    cur_count = 0
    for passage in data['passages']:
        cur_title = passage['title'].strip()
        passage['is_selected'] = 1 if cur_title in gt_titles else 0
    cur_count = sum([p['is_selected'] for p in data['passages']])
    if cur_count == 0:
        print(gt_titles)
        raise ValueError

    # if len(gt_documents) != len(data['answers']):
    #     print(len(gt_documents), len(data['answers']))
        # break
    count.append(cur_count)
    dureader_dataset_2.append(data)
sum(count)

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:00<00:00, 1722.56it/s]


362

In [ ]:
dureader_dataset_2[0]

{'input': '热诚传说结局',
 'context': '文章1\n标题：对结局好失望_热诚传说x吧_百度贴吧\n我想看的不是百合搞基番!为什么看公主和男主在一起,等了这么久的大结局,然后……你给我看这个?感觉前面都白看了,早知道不知结局了,前面25集都挺好的!看到大结局好失落,女主们最终还是没有第到使雷回来,新的道师也出现了,而男主醒来应该是几百年后了(城市都现代化了)!公主早就死了几百年了吧 不楼主请仔细看 新的导师就是史雷 史雷醒来应该也是几年后的事情 然而她们都长大了 下载贴吧APP看高清直播、视频! 贴吧页面意见反馈 违规贴吧举报反馈通道 贴吧违规信息处理公示 内容:\n\n文章2\n标题：《热诚传说》动画的结局是百合??? - 動漫區 - 百合会 - Powered by \n您需要 登录 才可以下载或查看,没有帐号?成为会员 四月百合淡季什么的,不存在的,顺便快去看re:creator最新一集 真的吗?真的吗!快来人打醒我 补特定几集就行了,整部作品来说没有明显的感情线,大家之间的距离都保持的很好,最后倒是真官方盖章百合,推得丧心病狂 这结局真喷了。。。但是为了百合补这番还是算了吧,不值得 ……好像可以借着这个结局坑别人玩一波游戏原作了。 参与人数 1积分+1 收起 理由 坟头纳彩 + 1 稳!“公主呢?” 查看全部评分 求补番指南,打算挑着看一看 您需要登录后才可以回帖 登录 | 成为会员\n\n文章3\n标题：【剧透】通关剧情分析吐槽 - 热情传说 - A9VG电玩部落论坛\n精华 0 帖子 1006 威望 0点 积分 1089 点 注册时间 2005-6-12 最后登录 2017-6-9 精华 0 帖子 8 威望 0点 积分 8点 注册时间 2014-9-7 最后登录 2017-3-16 表示无比赞同!玩了后说不出的感觉,就是感觉稀里糊涂+平淡无奇,说差也不差,但也说不上好。本来传说系列的一大特色就是充满矛盾、激情和冲突的剧情,到后期或结局往往令人心潮澎湃或者感动不已,远的如永恒传说,宿命传说2,近的如TOX。蛮失望的,不过会继续追传说系列! 您需要登录后才可以回帖 登录 | 注册\n\n文章4\n标题：热诚传说X 剧情介绍(全集大结局)_穿帮网\nchecking... 概述 简介 海报 我来发布 2016年放送開始!制作はufotab

In [ ]:
import pickle
with open('output/longbench_dureader.pkl', 'wb') as f:
    pickle.dump(dureader_dataset_2, f)
    f.close